**Part 4. Matrix and Vector computations**

13. Read  about  Broadcasting with  Arrays  onthe  chapter Computation  on  Arrays: Broadcastingfrom Python Data Science Handbook (J. VandePlas, 2016):Link: https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html.

Keep in mind when broadcasting:

*Memory Efficiency*: While broadcasting improves computational efficiency, it does not consume additional memory for duplicating array elements. The concept of stretching or duplicating is mainly a mental model to understand how the operations work.

*Compatibility*: Arrays need to follow broadcasting rules to be compatible. If the shapes of arrays cannot be aligned according to broadcasting rules, an error will be raised.

*Complexity*: Broadcasting becomes more complex with higher-dimensional arrays or when combining arrays of different dimensions. Understanding the broadcasting rules is essential to use it effectively and avoid errors.

14. Read Rewriting the particle simulator in NumpyonChapter 2: Fast Array Operations with  Numpy  and  Pandas(pp. 68)  from  the  book  G.  Lenaro  (2017).  Python  high Performance.   Second   Edition.   UK:   Packt   Publishing   Ltd.Implement   the improvements  on  the  particle  simulator  using  NumPy.  Show  that both implementations scale linearly with particle size, but the runtime in the pure Python versiongrows much faster than the NumPy version.

15. Explain how to optainthe optimal performance with numexpr. Read the section Reaching optimal performance with numexpr, pp. 72 from the previous reference. Implement it and measure the execution time.

In [29]:
import numpy as np
from timeit import timeit
from random import uniform
import numexpr as ne 

In [30]:
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel

class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)

        for _ in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])

        for _ in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i

        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

    def evolve_numexpr(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])

        for i in range(nsteps):
            norm_i = ne.evaluate("sqrt((r_i ** 2).sum(axis=1))")
            v_i = r_i[:, [1, 0]]
            v_i[:, 0] = ne.evaluate("-v_i[:, 0]")
            v_i = ne.evaluate("v_i / norm_i[:, None]")
            d_i = ne.evaluate("timestep * ang_vel_i[:, None] * v_i")
            r_i = ne.evaluate("r_i + d_i")

        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

def benchmark(npart=100, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
 
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)
    elif method == 'numexpr':
        simulator.evolve_numpy(0.1)

# Example of benchmarking
if __name__ == "__main__":
    
    print("Benchmarking with 1000 particles:")
    print("Python version:", timeit("benchmark(100, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(100, 'numpy')", globals=globals(), number=1), "seconds")

Benchmarking with 1000 particles:
Python version: 0.7236123000038788 seconds
NumPy version: 0.15063819999340922 seconds


Lets step it up a bit and make the total amount of particles to 1000 and watch the python version **increase** massively in execution time 

In [31]:
# Example of benchmarking
if __name__ == "__main__":

    print("Benchmarking with 1000 particles:")
    print("Python version:", timeit("benchmark(1000, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(1000, 'numpy')", globals=globals(), number=1), "seconds")

Benchmarking with 1000 particles:
Python version: 7.719938899972476 seconds
NumPy version: 0.3337502999929711 seconds


Increasing the amount of particles to **10,000** and comparing the **Numexpr** implementation with the other versions

In [34]:
if __name__ == "__main__":
    from timeit import timeit
    
    print("Benchmarking with 10000 particles:")
    print("Python version:", timeit("benchmark(10000, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(10000, 'numpy')", globals=globals(), number=1), "seconds")
    print("NumExpr version:", timeit("benchmark(10000, 'numexpr')", globals=globals(), number=1), "seconds")

Benchmarking with 10000 particles:
Python version: 79.07073779997882 seconds
NumPy version: 2.6994015999953263 seconds
NumExpr version: 2.1525922000291757 seconds
